In [ ]:
import sys, os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# NPTFit modules
from NPTFit import create_mask as cm

cur_dir = '/tigress/somalwar/SubhalosFresh/'
sys.path.append(cur_dir)
sys.path.append(cur_dir+"/Code/")
# My Functions
import PowerLaw
from AssortedFunctions import getEnergyBinnedMaps, myLog

In [ ]:
nside=128
my_iebins = [10, 15, 20, 40]
exposure_ebins = getEnergyBinnedMaps('/maps/exposure', cur_dir, my_iebins, ave=True, int32=False, nside=nside)

pscmask = np.array(np.load(cur_dir+'/fermi_data/fermidata_pscmask.npy'), dtype=bool)
mask = cm.make_mask_total(band_mask = True, band_mask_range = 5, mask_ring = True, inner = 20, outer = 180, custom_mask = pscmask)
area_rat = (len(mask[~mask]))/len(mask)

subhalos = np.load(cur_dir+'/fermi_data/EinastoTemplate.npy')
subhalos = subhalos/np.mean(subhalos)


In [ ]:
def convert_SCD_to_powerlaw_form3(best_fit_params):
    best_fit_params[0] = 10**best_fit_params[0]
    best_fit_params[-2] = best_fit_params[-3]*best_fit_params[-2]
    best_fit_params[-1] = best_fit_params[-2]*best_fit_params[-1]
    best_fit_params = np.array(best_fit_params*np.array([np.average(exposure_ebins[ib][~mask]/area_rat/(4*np.pi * (180/np.pi)**2)*np.sum(subhalos[~mask])), 1., 1., 1., 1., 1/np.average(exposure_ebins[ib][~mask
    ]), 1/np.average(exposure_ebins[ib][~mask]), 1/np.average(exposure_ebins[ib][~mask])]))
    best_fit_params[0] = np.log10(best_fit_params[0])
    best_fit_params[-3:] = np.log10(best_fit_params[-3:])
    best_fit_params[-1] = np.abs(best_fit_params[-2]) + best_fit_params[-1]
    best_fit_params[-2] = np.abs(best_fit_params[-3]) + best_fit_params[-2]
    return best_fit_params

def convert_SCD_to_powerlaw_form2(best_fit_params):
    best_fit_params[0] = 10**best_fit_params[0]
    best_fit_params[-1] = best_fit_params[-2]*best_fit_params[-1]
    best_fit_params = np.array(best_fit_params*np.array([np.average(exposure_ebins[ib][~mask]/area_rat/(4*np.pi * (180/np.pi)**2)*np.sum(subhalos[~mask])), 1., 1., 1., 1/np.average(exposure_ebins[ib][~mask]), 1/np.average(exposure_ebins[ib][~mask])]))
    best_fit_params[0] = np.log10(best_fit_params[0])
    best_fit_params[-2:] = np.log10(best_fit_params[-2:])
    best_fit_params[-1] = np.abs(best_fit_params[-2]) + best_fit_params[-1]
    return best_fit_params

In [ ]:
F_test = np.logspace(-12,6,5000)
ib = 0
best_fit_params = np.load(cur_dir+"/SCD/blazSCD_iebins-15to20_0.npy")
print(best_fit_params)
print(best_fit_params[0])
best_fit_params_med = convert_SCD_to_powerlaw_form2(best_fit_params)
plt.plot(myLog(F_test), 2*myLog(F_test)+PowerLaw.simple_power(myLog(F_test), False, *best_fit_params_med), zorder=5, linewidth=3, c="blue", alpha=0.7, label="NFW, Point Source") 